In [3]:
from gensim.models import Word2Vec
import numpy as np
from keras.preprocessing.text import text_to_word_sequence

In [92]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [4]:
def create_sentences_from_files(pos_path,neg_path):
    sentences=list()
    with open(pos_path) as f:
        for  line in f:
            sentences.append(line)
    with open(pos_path) as f2:   
        for line in f2:
            sentences.append(line)
    return np.array(sentences)        

In [5]:
sentences=create_sentences_from_files('twitter-datasets/train_pos.txt','twitter-datasets/train_neg.txt')

In [6]:
def create_tokenized_sentences(sentences):
    X=list()
    for sentence in sentences:
        X.append(text_to_word_sequence(sentence,filters=''))
    return X


#sentences[0]

#tokens=np.array(np.array(text_to_word_sequence(xi)) for xi in sentences )
#np.shape(tokens)
#print(tokens)


#squarer = lambda t: text_to_word_sequence(t) 
#vfunc = np.vectorize(squarer)
#vfunc(sentences)

In [27]:

tokenized=create_tokenized_sentences(sentences)


In [28]:
def create_word2vec_model(sentences_tokenized):
    window_size = 5
    size = 50
    epochs = 100
    min_count = 1
    sg = 1
    
    model = Word2Vec(
        sentences_tokenized,
        sg=1,
        window=5,
        size=size,
        min_count = min_count,
        iter=epochs,
        sample=0.00001)
    model.save("word2vec.model")
            
            

In [29]:
create_word2vec_model(tokenized)

In [30]:
model = Word2Vec.load("word2vec.model")

In [83]:
def create_word2vec_features(sentences, model ,dim):
    X=[]
    not_invocab=[]
    for line in sentences:
        sum_embedding=np.zeros(dim)
        nb_words=0
        words=line.split()
        for word in words:
            if word in model.wv.vocab:
                sum_embedding+=model.wv[word]
                nb_words+=1
            else:
                not_invocab.append(word)
        if (nb_words!=0):
            X.append(sum_embedding/nb_words)
        else:
            X.append(sum_embedding)
    Y=np.zeros(len(sentences))
    Y[:100000]=0
    Y[100000:]=1
    return np.squeeze(X),Y,not_invocab  
        
        

In [84]:
X,Y,K=create_word2vec_features(sentences, model ,50)
#% store X

In [87]:
random_idxs = np.random.permutation(len(Y))
X = X[random_idxs]
Y = Y[random_idxs]

N_train = int(0.8*len(Y))

X_train, X_test = X[:N_train], X[N_train:]
y_train, y_test = Y[:N_train], Y[N_train:]

In [88]:
clf = make_pipeline(StandardScaler(), linear_model.SGDClassifier(loss = 'log', max_iter=1000, tol=1e-4, verbose = True ))


clf.fit(X_train, y_train)
(clf.predict(X_test) == y_test).mean()

-- Epoch 1
Norm: 2.19, NNZs: 50, Bias: 0.189958, T: 160000, Avg. loss: 4.029196
Total training time: 0.07 seconds.
-- Epoch 2
Norm: 1.17, NNZs: 50, Bias: -0.041819, T: 320000, Avg. loss: 0.924074
Total training time: 0.14 seconds.
-- Epoch 3
Norm: 0.89, NNZs: 50, Bias: -0.148589, T: 480000, Avg. loss: 0.807583
Total training time: 0.21 seconds.
-- Epoch 4
Norm: 0.77, NNZs: 50, Bias: -0.002517, T: 640000, Avg. loss: 0.767933
Total training time: 0.29 seconds.
-- Epoch 5
Norm: 0.59, NNZs: 50, Bias: -0.003374, T: 800000, Avg. loss: 0.748216
Total training time: 0.36 seconds.
-- Epoch 6
Norm: 0.57, NNZs: 50, Bias: -0.010173, T: 960000, Avg. loss: 0.736570
Total training time: 0.43 seconds.
-- Epoch 7
Norm: 0.51, NNZs: 50, Bias: -0.006453, T: 1120000, Avg. loss: 0.728445
Total training time: 0.51 seconds.
-- Epoch 8
Norm: 0.50, NNZs: 50, Bias: 0.032735, T: 1280000, Avg. loss: 0.723504
Total training time: 0.58 seconds.
-- Epoch 9
Norm: 0.50, NNZs: 50, Bias: 0.061374, T: 1440000, Avg. loss: 

0.500825

In [ ]:

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', tol = 1e-3, verbose = True, max_iter = 10000))

clf.fit(X_train, y_train)
(clf.predict(X_test) == y_test).mean()

[LibSVM]

In [94]:
clf = make_pipeline(StandardScaler(), MLPClassifier(solver='lbfgs', alpha=1e-2, \
                                                    hidden_layer_sizes=(10, 5), random_state=0, verbose = True, max_iter = 1000))

clf.fit(X_train, y_train)
(clf.predict(X_test) == y_test).mean()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.5005

In [91]:
clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5, loss = 'squared_hinge', dual = False))
clf.fit(X_train, y_train)
(clf.predict(X_test) == y_test).mean()

NameError: name 'LinearSVC' is not defined

In [ ]:
%store X

In [33]:
#% store -r X

In [34]:
np.shape(X)

(200000, 50)

In [35]:
print(K)

['#crakkbitch', 'x15', 'katemelo', 'fairest', '#reasons2dothebirdmanhandrub', 'bbs', 'cracow', '#stupidhashtag', '#realthug', 'banglorean', '#joketweet', '#yess', '#imintoyou', '#wale', 'tele', '#hittheroadjack', 'staage', '#kindalethal', '#fuckswithme', '#loveeeyou', '#loveeeyou', '#no', '#hamoswant1dtour', '#hamoswant1dtour', 'wasttteddd', '#scotlandaintready', '#goodluckcharlieseason3premiere', 'horoscopes', 'shagger', 'shagger', '#imlimitededition', '#consultant', 'lauri', '#reallythough', '#aheartforgod', '#obb', '#somemovie', 'ruschers', '#noitsnotfordrugsoranythingbad', '#applebutt', '#whatupwiththat', '#teamleo', '#soyouknowitsreal', '#newtothis', '#cantbearblair', 'maica', 'tybfr', '#lucyyoung', '#rookiemistakes', 'arlene', 'sunshineee', '#finallyyouadmitit', 'furnitures', '#teamdabr', '3bp3', 'alreadyyy', 'sexaaay', '#letshopeforthebest', '#jenny_meszaros', 'openfollow', 'chirping', 'entrenovias', '#ourwholeuniversewasinahotdensestate', '#myjam', 'selfharm', '#stepbrothers', 

In [36]:
len(model.wv.vocab)

63974